In [1]:
import pandas as pd
import db_access
import math

In [2]:
def get_raw_data(query):
    engine = db_access.get_engine()
    return pd.read_sql(query, engine)

In [3]:
query = "select distinct * from usd_jpy_4h where datetime between '2019/09/01' and '2020/03/01' order by 1"

df = get_raw_data(query)
df = df[["datetime","open","high","low","close","volume"]].copy()
df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.reset_index().set_index('Datetime')
df.head()

,index,Open,High,Low,Close,Volume
Datetime,,,,,,
2019-09-01 20:00:00,0,106.059,106.168,105.932,106.136,2195
2019-09-02 00:00:00,1,106.138,106.242,106.078,106.157,2294
2019-09-02 04:00:00,2,106.154,106.300,106.144,106.256,1510
2019-09-02 08:00:00,3,106.259,106.400,106.244,106.352,1713
2019-09-02 12:00:00,4,106.354,106.388,106.108,106.202,1642


In [4]:
df.head(30)

,index,Open,High,Low,Close,Volume
Datetime,,,,,,
2019-09-01 20:00:00,0,106.059,106.168,105.932,106.136,2195
2019-09-02 00:00:00,1,106.138,106.242,106.078,106.157,2294
2019-09-02 04:00:00,2,106.154,106.300,106.144,106.256,1510
2019-09-02 08:00:00,3,106.259,106.400,106.244,106.352,1713
2019-09-02 12:00:00,4,106.354,106.388,106.108,106.202,1642
2019-09-02 16:00:00,5,106.200,106.205,106.058,106.202,964
2019-09-02 20:00:00,6,106.200,106.259,106.171,106.195,1663
2019-09-03 00:00:00,7,106.197,106.385,106.197,106.302,2728
2019-09-03 04:00:00,8,106.306,106.347,105.921,105.986,3988


In [5]:
df.tail()

,index,Open,High,Low,Close,Volume
Datetime,,,,,,
2020-02-28 04:00:00,791,109.144,109.188,108.792,108.856,44192
2020-02-28 08:00:00,792,108.854,108.980,108.510,108.692,51942
2020-02-28 12:00:00,793,108.694,108.870,107.754,107.931,65606
2020-02-28 16:00:00,794,107.930,108.501,107.761,107.860,73602
2020-02-28 20:00:00,795,107.860,108.140,107.512,108.064,15980


In [6]:
import talib as ta

In [7]:
def get_bband(close, timeperiod=20, nbdevup=3, nbdevdn=3, matype=0):
    gain = pd.DataFrame(close)
    gain.columns = ['close']
    upper, middle, lower = ta.BBANDS(gain.close, timeperiod, nbdevup , nbdevdn, matype)
    gain['bb_upper'] = upper
    gain['bb_lower'] = lower
    return gain['bb_upper'], gain['bb_lower']

In [8]:
def get_macd(close, fastperiod=6, slowperiod=13, signalperiod=4):
    macd, macdsignal, macdhist = ta.MACD(close, fastperiod=fastperiod, slowperiod=slowperiod, signalperiod=signalperiod)
    return macd, macdsignal, macdhist

In [9]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

/Users/kataoka/opt/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:44: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [10]:
class MacdStrategy(Strategy):
    macd_fastperiod = 6
    macd_slowperiod = 13
    macd_signalperiod = 4
    profit_pips = 50
    loss_pips = 50
    profit = profit_pips * 0.01
    loss = loss_pips * 0.01
    hist_seps = 2
    day_seps = 6
    
    def init(self):
        self.macd, self.signal, self.hist = self.I(get_macd, self.data.Close, self.macd_fastperiod, self.macd_slowperiod, self.macd_signalperiod)
        
        # 乖離してからのロウソク数
        self.seps = 0

    def next(self):
        # 乖離確認
        if (math.fabs(self.hist) * 100 > self.hist_seps):
            self.seps = self.day_seps

        if (self.seps > 0):
            if (crossover(self.signal, self.macd)):
                self.buy(sl=self.data.Close[-1] - self.loss, tp=self.data.Close[-1] + self.profit)
            elif (crossover(self.macd, self.signal)):
                self.sell(sl=self.data.Close[-1] + self.loss, tp=self.data.Close[-1] - self.profit)
            else:
                self.seps -= 1


In [11]:
bt = Backtest(df, MacdStrategy, cash=100000, commission=.00004)
output = bt.run()
print(output)

Start                     2019-09-01 20:00:00
End                       2020-02-28 20:00:00
Duration                    180 days 00:00:00
Exposure [%]                          74.9074
Equity Final [$]                        98647
Equity Peak [$]                        101404
Return [%]                           -1.35297
Buy & Hold Return [%]                 1.81654
Max. Drawdown [%]                     -2.7724
Avg. Drawdown [%]                   -0.518637
Max. Drawdown Duration      162 days 00:00:00
Avg. Drawdown Duration       21 days 18:00:00
# Trades                                   66
Win Rate [%]                               50
Best Trade [%]                       0.457344
Worst Trade [%]                      -0.68218
Avg. Trade [%]                     -0.0215735
Max. Trade Duration          10 days 00:00:00
Avg. Trade Duration           2 days 02:00:00
Expectancy [%]                       0.275169
SQN                                 -0.538945
Sharpe Ratio                      

In [15]:
stats = bt.optimize(
    macd_fastperiod = [6, 8, 10, 12],
    macd_slowperiod = [13, 19, 26],
    macd_signalperiod = [4, 7, 9],
    profit_pips = [30, 50, 70, 90],
    loss_pips = [20, 30, 40, 50],
    hist_seps = [1, 2],
    day_seps = [6, 8, 10, 12]
)

/Users/kataoka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Searching best of 4608 configurations.
  


In [16]:
print(stats)

Start                                                   2019-09-01 20:00:00
End                                                     2020-02-28 20:00:00
Duration                                                  180 days 00:00:00
Exposure [%]                                                        40.9259
Equity Final [$]                                                     102216
Equity Peak [$]                                                      102521
Return [%]                                                          2.21604
Buy & Hold Return [%]                                               1.81654
Max. Drawdown [%]                                                  -1.37921
Avg. Drawdown [%]                                                  -0.31035
Max. Drawdown Duration                                     48 days 12:00:00
Avg. Drawdown Duration                                      9 days 03:00:00
# Trades                                                                 18
Win Rate [%]

In [17]:
print(stats._strategy)

MacdStrategy(macd_fastperiod=10,macd_slowperiod=13,macd_signalperiod=9,profit_pips=30,loss_pips=20,hist_seps=2,day_seps=6)
